# Memory 
## An explanation of the EDA

### Original tables:

- #### add_val_ind_EU27 (added value by industry and country in the EU 27)
    - ##### **Columns**:
        - ##### **units** (Current prices, million euro )
        - ##### **nace_r2** (industry types)
        - ##### **na_item** ( gross value added, Compensation of employees , Operating surplus and mixed income, net )
        - ##### **geo** (countries)
        - ##### **TIME_PERIOD** (in years)
        - ##### **OBS_VALUE** (values)

- #### Co2_eu27 (emission by industry and country in EU 27)
    - ##### **Columns**:
        - ##### **airpol** (type of emissions)
        - ##### **nace_r2** (industry types)
        - ##### **unit** (tonnes)
        - ##### **geo** (countries)
        - ##### **TIME_PERIOD** (in years)
        - ##### **OBS_VALUE** (values)

- #### Eu_27_GDP (GDP by country in EU 27)
    - ##### **Columns**:
        - ##### **units** (Current prices, million euro )
        - ##### **na_item** (CP_MEUR Current prices, million euro)
        - ##### **geo** (countries)
        - ##### **TIME_PERIOD** (in years)
        - ##### **OBS_VALUE** (values)

- #### EU27_ener_ind (Energy consumtion by industry and country)
    - ##### **Columns**:
        - ##### **nrg_bal** (industry types, in bigger groups)
        - ##### **siec** (energy type, only total energy)
        - ##### **units** (Gigawatt hour)
        - ##### **geo** (countries)
        - ##### **TIME_PERIOD** (in years)
        - ##### **OBS_VALUE** (values)


### Bridge Tables:

- #### country_table:
    - ##### **columns**:
        - ##### **country_code** (original code)
        - ##### **country_name** (name country)

- #### energy_industry_table:
    - ##### **columns**:
        - ##### **sectors_code**
        - ##### **sectors_name**
        - ##### **small_name** (shorter name version for visuals)

- #### industry_table:
    - ##### **columns**:
        - ##### **industry_code**
        - ##### **industry**

### Tables in Power BI:

#### Code in Dax:
````
gas_name = DISTINCT(SELECTCOLUMNS(Co2_eu27,"gas",Co2_eu27[airpol]))

gas emissions = if(gas_name[gas]=="CO2", "CO2","CH4_EQU")
````
- #### gas_name:
    - ##### **gas_name**
    - ##### **gas_emission**

#### Code in Dax:
````
Time_Table = DISTINCT(ALLSELECTED(add_val_ind_EU27[TIME_PERIOD]))
````
- #### Time_Table:
    - ##### **columns**:
        - ##### TIME_PERIOD

### TIME PREDICTION TABLE

#### Code in Dax:

````
TIME_PERIOD = GENERATESERIES(1980, 2050, 1)
````

- #### TIME_PERIOD:
    - ##### **columns**:
        - ##### TIME_PERIOD


## Model:

### Now that we know what tables we will be working with we can take a look at the model:
![Model](./Model.PNG)

As we can see the bridge tables, country_table and time_table, are connected with all tables as all tables have those variables.  
The industry_table is only connected with add_val_ind_EU27 and Co2_eu27.  
And both gas_name and energy_industry_table are only connected to one each, Co2_eu27 and EU27_ener_ind respectably.  
All created table are in a one to many relation in a single direction to avoid recursivity.  


## Measures in Power BI:

### %add_val_growth (this measure give the % of growth of value added from the first year to the last, used in the table in the GDP_val_add page):
````
var tab_1 = SUMMARIZE(add_val_ind_EU27,add_val_ind_EU27[TIME_PERIOD],"sum",sum(add_val_ind_EU27[OBS_VALUE]))
var tab_2=FILTER(tab_1,add_val_ind_EU27[TIME_PERIOD]==min(add_val_ind_EU27[TIME_PERIOD]))
var max_1=maxx(tab_2,[sum])
var tab_3=filter(tab_1,add_val_ind_EU27[TIME_PERIOD]==max(add_val_ind_EU27[TIME_PERIOD]))
var min_1=maxx(tab_3,[sum])
var cal= (min_1-max_1)/max_1*100
return
cal
````
### last_add_val (this calculates the value added of the last year, used in the table in the GDP_val_add page):
````
last_add_val = 
CALCULATE(sum(add_val_ind_EU27[OBS_VALUE]),add_val_ind_EU27[TIME_PERIOD]=max(add_val_ind_EU27[TIME_PERIOD]))
````

### %grown_GWH (this measure give the % of growth of GWH from the first year to the last, used in the table in GWH_consumption page):
````
%grown_GWH = 
var first=CALCULATE(sum(EU27_ener_ind[OBS_VALUE]),EU27_ener_ind[TIME_PERIOD]==MIN(EU27_ener_ind[TIME_PERIOD]))
var growth=([last_gwh]-first)/first*100
return 
growth
````
### last_gwh (this calculates the consumption of energy in GWH of the last year, used in the table in the GWH_consumption page ):
````
last_gwh = CALCULATE(sum(EU27_ener_ind[OBS_VALUE]),EU27_ener_ind[TIME_PERIOD]==max(EU27_ener_ind[TIME_PERIOD]))
````
### %growth_CO2 (this measure give the % of growth of CO2 and CH4 equivalent emission from the first year to the last, used in the table in CO2_by country page):
````
%growth_CO2 = 
var first=calculate(sum(Co2_eu27[OBS_VALUE]),Co2_eu27[TIME_PERIOD]==MIN(Co2_eu27[TIME_PERIOD]))
var growth=([last_CO2]-first)/first*100
return
growth
````
### last_CO2 (this calculates the CO2 and CH4 equivalent emission of the last year, used in the table in the CO2_by country page ):
````
last_CO2 = calculate(sum(Co2_eu27[OBS_VALUE]),Co2_eu27[TIME_PERIOD]==max(Co2_eu27[TIME_PERIOD]))
````
### ADD_VALUE/CO2_ratio (this calculates the ratio of value added per CO2 emission, and can be seen in the page Value_added_by industry in both graphs):
````
ADD_VALUE/CO2_ratio = 
var cal_1=CALCULATE(sum(Co2_eu27[OBS_VALUE]))
var cal_2=CALCULATE(sum(add_val_ind_EU27[OBS_VALUE]))
var ratio=cal_2/cal_1
return
if (or(cal_1==0,cal_2==0),0 ,ratio*1000000)
````
### country_name_slice (this is use for the first 3 pages to give a title to all the tables):
````
country_name_slice = if(SELECTEDVALUE(country_table[country_name])==BLANK(),"Euro27",SELECTEDVALUE(country_table[country_name]))
````

because there are several ways of measuring the same thing in particular in economic variable I use Power BI's filter side bar for this purpose:  
  
![filter](./side_bar.PNG)  
  
as we can see in this image the na_item is B1G, this is value added, that way the measures would work as intended

## Linear Regressions

To make linear regressions in Power BI is important to truly understand this calculation, because one need to calculate each value:  
The formula for a line goes like so: y = mx + b  
Where:  
  
**y** is what we want to predict  
  
**m** is the slope i.e. the change in y divided by change in x represented this way:  
  
![m](./m.PNG)  
  
**x** is the value we give in theses cases will be years  
  
**b** is the value of y when x is 0 calculated like so:  
  
![b](./b.PNG)  
  


As we can see we will need 2 values that are not in the original data:
- $X^2$
- $XY$ 
  
Create $X^2$ column for EU_27_GDP, Co2_eu27 and EU27_ener_ind
````
X^2 = tables_name[TIME_PERIOD]^2
````
Create $XY$ column for EU_27_GDP, Co2_eu27 and EU27_ener_ind
````
XY = tables_name[TIME_PERIOD]*tables_name[OBS_VALUE]
````  
  
now that we have the $X^2$ and $XY$ we need measurements to calculate the sum of:
- $X$
- $Y$
- $XY$
- $X^2$

### Measurements

#### SUMX
````
SUMX = sum(tables_name[TIME_PERIOD])
````
#### SUMY
````
SUMX = sum(tables_name[OBS_VALUE])
````
#### SUMXY
````
SUMX = sum(tables_name[XY])
````
#### SUMX^2
````
SUMX = sum(tables_name[X^2])
````
  
we also need n, this is the number of observation

#### N
````
N=countrows(tables_name)
````

now that we have all the components we can make m and b as measurements as well:

#### M 
````
M = DIVIDE([N]*[SUMXY]-[SUMX]*[SUMY],[N]*[SUMX^2]-[SUMX]^2,0)
````
  
#### B
````
B = DIVIDE([SUMY]*[SUMX^2]-[SUMX]*[SUMXY],[N]*[SUMX^2]-[SUMX]^2,0)
````
  
And finally we group all of this measurements to make a line equation:
  
#### LINEAR_REGRESSION
````
LINEAR_REGRESSION = [M]*(TIME_PERIOD[TIME_PERIOD Value])+[B]
````
  
because TIME_PERIOD table has years up to 2050 we can make future prediction uo to 2050.  

### Extra Measurements

#### 2050 predicted value for GDP GWH and CO2
````
choosen_value_2050 = [M]*(2050)+[B]
````
  
#### growth for 2050 for GDP GWH and CO2
````
%_growth = 
var first=CALCULATE(sum(tables_name[OBS_VALUE]),tables_name[TIME_PERIOD]==min(tables_name[TIME_PERIOD]))
var cal=(tables_name[choosen_value_2050]-first)/first
return
cal
````
#### total emissions by 2050
````
emission_2050 = 
VAR co2=CALCULATE([CO2_2050],and(Co2_eu27[airpol]=="CO2",Co2_eu27[nace_r2]=="TOTAL"))
VAR ch4=CALCULATE([CO2_2050],and(Co2_eu27[airpol]=="CH4_CO2E",Co2_eu27[nace_r2]=="TOTAL"))
RETURN
co2+ch4
````

#### ratio Measurements

##### GDP/CO2_2050
````
GDP/CO2_2050 = [GDP_2050]/[emission_2050]*1000000
````
  
##### GDP/GWH_2050
````
GDP/GWH_2050 = [GDP_2050]/[GWH_2050]
````
  
##### GWH/CO2_2050
````
GWH/CO2_2050 = [GWH_2050]/[emission_2050]
````
